## Estimating the marginal effect of NO~2~ on ED Visits using CTMLE

I want to know the marginal effect of NO~2~ > 19 on I have high-dimension data. I was peforming targeted maximum likelihood estimate in R using the TMLE package, but I was having trouble finding covariates to adjust for and results from TMLE due to ETA violations.

Enter Collaborative TMLE (CTMLE) that is specifically designed to tackle this problem. Unforunately there is no R package to perform CTMLE, but there is a Julia package. So here I am: using a Julia kernel, in Jupyter notebooks. 

Tutorial here: https://github.com/lendle/TargetedLearning.jl

### Reading in NO2 ED visit dataframe

In [22]:
# call dataframes and csv packages
using DataFrames, CSV

**Loading Package "GLM"**

Added glm package using *Pkg.add("GLM")*

In [38]:
using GLM

INFO: Precompiling module GLM.


#### NO2 Dataframe

I think the GLM function needs a dataframe and not an array. Going to create dataframe of no2.

In [56]:
# no2_df
no2_df = CSV.read("./asthma_no2_julia.csv")

,edvisit,no2_cut,no2_med,male,enghome,spanhome,api,afra,latino,mixed,othrace,dtchres,mixhouse,urbanres,pwhite,pblack,paina,ppi,pmulti,pctlatw,pctlatb,pctlataian0,pctlato,pctmultil,pltsch,psch,padult,psenior,pmale,pnfhhd1,pinst,pmchhwoc,pfhhwc,pmhhwoc,pmccu18,pmhhcu18,pfamhhle4,pnfhhle4,pnfhhgt4,pmcfcu6,pmcfschk,pmhhku6,pmhhschk,pfhhku6,pfhhschk,pnowmarr,pnevmarr,pdiv,pwidow,pmarrspp,kidspanengn,kidieengg,kidasnengg,kidasnengn,kidolengg,adspanengg,adieengg,adieengn,adasnengg,adasnengn,adolengg,srengonly,srspanengg,srspanengn,srieengg,srasnengg,srasnengn,eslgood,pctcal,pnotctzn,pimgrlt10y,pimgr10_20,pimgrgt20,pimm00_00,pimm00_00nc,pimm80_89,pimm80_89nc,pimmb80,pimmb80nc,psamehome95,psamecty95,psamest95,pctdrove,pctpubtr,pworkhome,pcommlt30min,pctk_8pub,pctk_8priv,pcthspub,pcthspriv,pcths,pctgrad,pdischild,pdissr,pctemp,pctnotlf,p2parlf1,p2parlf2,p1parlf,p2parnlf,psplfwk,psplfun,pspnlf,pfemwclt18,pmomnlf,pklt62p1,pklt62p2,pklt62p3,pklt6sp1,pklt6sp2,pkgt62p1,pkgt62p2,pkgt62p3,pkgt6sp1,pkgt6sp2,phlt25k,ph25_50k,ph50_75k,ph75_100k,phgt100k,phhdwage,phhdwssi,phhdpubast,phhdwretinc,agssi_t,agpubast_t,ag_ret_m,pf25_50k,pf50_75k,pf75_100k,agfaminc_m,pci_t,pkidspov,padpov,psrpov,piprlt50,pipr100_150,pfampov,pmckpov,pspkpov,agincdefmc,agincdefsp,pctgrand,pgrandguar,mhi_wh_t,mhi_bl_t,mhi_asn_t,mhi_lat_t,houseunits,pctvac,pct4rent,pct4sale,pctaianhhd,pctpihhd,pctmulthhd,pctwown,pctaiown,pctasown,pctotown,pctmuown,pctwrent,pctairent,pctasrent,pctpirent,pctmurent,avgsizeown,avgsizerent,medrooms,pshouseun,pshouseat,pbld2_4,pbld5_9,pbld10_19,pbld20_49,pbldgt50,medyrblt,medbltown,medbltrent,pctelec,pctwood,pctnofuel,pct0br,pct1br,pct2br,pct3br,pct4plbr,pownnoph,prentnoph,pnoplumb,pnokitch,pctutilinc,rentpctinc,mhomeval_t,aggrent_t,medrent_h,asstct,pubhct,x_pubh,x_hwy,x_inter,x_sthwy,hwytypes,x_hillres,x_detres,x_mixedhouse,x_urbanres,x_neighctr,x_commcomr,x_hsebusmix,x_regcomm,x_busmix,x_industry,x_instit,x_rescon,x_urbanopen,totaluse,racemaj,whitemaj,blackmaj,asianmaj,latinomaj,majgroup,claremont,frick,bretharte,edbrewer,montera,roosevelt,westlake
1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,78.6932,5.15422,0.446429,0.0,3.53084,48.6726,4.42478,0.0,21.2389,25.6637,5.11364,9.21266,68.0195,17.6542,50.6899,49.2901,0.0,59.7015,5.69878,0.949796,42.1338,1.80832,62.4198,32.4473,0.0,17.9104,21.1669,0.814111,0.542741,0.0,0.678426,64.4231,21.7033,9.66117,4.21245,96.3042,0.0,2.64317,10.5727,0.0,1.76211,5.31026,6.50358,0.298329,6.02625,0.0,0.536993,91.2644,3.90805,0.0,3.21839,0.0,0.0,16.3815,37.8247,41.7874,23.6715,9.90338,66.4251,27.2947,100.0,15.2174,46.0317,57.4879,13.0252,56.8862,15.0984,17.8358,80.8784,8.37838,8.91892,58.5312,18.0617,44.0529,9.2511,28.6344,3.8254,52.7219,0.0,57.0064,99.0053,30.5069,51.8317,0.0,26.73,12.7544,4.6133,0.678426,3.39213,21.6446,27.9476,22.2857,11.7143,2.85714,1.42857,0.0,47.7143,10.0,0.0,4.0,0.0,5.22456,11.9157,9.89917,9.1659,63.7947,78.1852,0.458295,0.641613,20.8983,42.0,160.5,6.2625,8.2768,10.9905,4.47761,144.792,75.97,13.8889,77.7778,8.33333,2.31613,2.55994,0.678426,100.0,0.0,50.1,0.0,3.21744,0.0,128.428,195.204,138.963,66.25,1151,5.31915,0.0,19.4444,0.538117,0.0,3.13901,83.3496,0.584226,9.44499,0.0,2.14216,59.0909,0.0,17.0455,0.0,14.7727,2.26,1.6,6.9,70.4605,24.4136,4.86533,0.260643,0.0,0.0,0.0,1991,1991,1956,9.95516,0.0,0.627803,0.0,2.95395,20.7646,38.9227,37.3588,0.681597,0.0,1.56386,1.30321,25.0,22.8,695.9,145.0,17.34,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,1,1,0,6,78.6932,1,0,0,0,1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,1,0,1,0,69.9467,13.5096,0.0,0.307503,6.25256,34.8592,0.0,0.0,44.0141,9.85915,6.80607,6.64207,77.5318,9.02009,47.8065,51.7751,100.0,38.4766,9.96094,4.58984,39.5996,4.44939,38.52,58.76,0.28,16.9922,20.6055,0.976563,2.92969,5.66406,3.61328,41.6492,42.0674,11.3589,4.92451,84.8299,0.0,2.77778,0.0,0.0,2.16049,6.82179,2.24749,0.76679,3.9926,0.0,5.12956,85.4545,1.59091,0.0,5.68182,2.04545,0.0,16.806,41.2054,62.7005,21.2567,38.6364,40.107,46.9251,85.755,21.2567,51.5723,31.8182,36.1345,46.3044,22.5253,15.3322,67.089,21.6316,4.66625,56

In [51]:
# simple logistic model
no2_logmod = glm(@formula(edvisit ~ no2_cut), no2_df, Binomial(), LogitLink())

StatsModels.DataFrameRegressionModel{GLM.GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Distributions.Binomial{Float64},GLM.LogitLink},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: edvisit ~ 1 + no2_cut

Coefficients:
             Estimate Std.Error  z value Pr(>|z|)
(Intercept)   -1.1995 0.0910428 -13.1752   <1e-38
no2_cut      0.569271  0.213696  2.66393   0.0077
